In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
dfElche = pd.read_csv('../Procesamiento/Elche-limpio.csv', sep=';')
dfOrihuela = pd.read_csv('../Procesamiento/Orihuela-limpio.csv', sep=';')
dfTorrevieja = pd.read_csv('../Procesamiento/Torrevieja-limpio.csv', sep=';')

df_combined = pd.concat([dfElche, dfOrihuela, dfTorrevieja], ignore_index=True)

df_combined.tail(10)

Veo la cantidad de nulos

In [ ]:
df_combined.isnull().sum()

Relleno las columnas con menos de mil nulos

In [ ]:
df_combined = df_combined.apply(lambda col: col.astype('category').cat.codes if col.dtypes == 'object' or col.dtypes.name == 'category' else col)
df_combined

In [ ]:
columnas_menos_nulos = df_combined.columns[df_combined.isnull().sum() < 400000]

for col in columnas_menos_nulos:
    df_combined[col].fillna(df_combined[col].median(), inplace=True)

In [ ]:
df_combined.isnull().sum()

Elimino las que tengan mas nulos 

In [16]:
nulos_por_columna = df_combined.isnull().sum()
columnas_mas_1000_nulos = nulos_por_columna[nulos_por_columna >= 400000].index

df_combined.drop(columns=columnas_mas_1000_nulos, inplace=True)

In [ ]:
df_combined.isnull().sum()

In [18]:
y = df_combined['SO2']
X = df_combined.drop(['SO2'],axis=1)

In [ ]:
y

In [ ]:
X

In [21]:
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
model.score(X_test,y_test)

Usando Dos algoritmos y viendo con graficas cada uno Automatizado

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

dfElche = pd.read_csv('../Procesamiento/Elche-limpio.csv', sep=';')
dfOrihuela = pd.read_csv('../Procesamiento/Orihuela-limpio.csv', sep=';')
dfTorrevieja = pd.read_csv('../Procesamiento/Torrevieja-limpio.csv', sep=';')

df_combined = pd.concat([dfElche, dfOrihuela, dfTorrevieja], ignore_index=True)

print(df_combined.tail(10))

print(df_combined.isnull().sum())

df_combined = df_combined.apply(lambda col: col.astype('category').cat.codes if col.dtypes == 'object' or col.dtypes.name == 'category' else col)

print(df_combined)

columnas_menos_nulos = df_combined.columns[df_combined.isnull().sum() < 400000]

for col in columnas_menos_nulos:
    df_combined[col].fillna(df_combined[col].median(), inplace=True)

print(df_combined.isnull().sum())

nulos_por_columna = df_combined.isnull().sum()
columnas_mas_1000_nulos = nulos_por_columna[nulos_por_columna >= 400000].index
df_combined.drop(columns=columnas_mas_1000_nulos, inplace=True)

print(df_combined.isnull().sum())

y = df_combined['SO2']
X = df_combined.drop(['SO2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf_model = RandomForestClassifier()
lr_model = LogisticRegression(max_iter=1000)


rf_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)


rf_accuracy = accuracy_score(y_test, rf_predictions)
lr_accuracy = accuracy_score(y_test, lr_predictions)


print(f"Precisión de RandomForest: {rf_accuracy * 100:.2f}%")
print(f"Precisión de LogisticRegression: {lr_accuracy * 100:.2f}%")

models = ['RandomForest', 'Logistic Regression']
accuracies = [rf_accuracy, lr_accuracy]

plt.figure(figsize=(8, 5))
sns.barplot(x=models, y=accuracies, palette='viridis')
plt.title('Comparación de Precisión de Modelos')
plt.xlabel('Modelo')
plt.ylabel('Precisión')
plt.ylim(0, 1)
plt.show()


Sacando el ml a un archivo pickle

Modelo con el target O3

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle


dfElche = pd.read_csv('../Procesamiento/Elche-limpio.csv', sep=';')
dfOrihuela = pd.read_csv('../Procesamiento/Orihuela-limpio.csv', sep=';')
dfTorrevieja = pd.read_csv('../Procesamiento/Torrevieja-limpio.csv', sep=';')


df_combined = pd.concat([dfElche, dfOrihuela, dfTorrevieja], ignore_index=True)

print(df_combined.tail(10))

print("\nValores nulos iniciales por columna:")
print(df_combined.isnull().sum())

if 'FECHA' in df_combined.columns:
    df_combined['FECHA'] = pd.to_datetime(df_combined['FECHA'], errors='coerce')

    df_combined['año'] = df_combined['FECHA'].dt.year
    df_combined['mes'] = df_combined['FECHA'].dt.month
    df_combined['dia'] = df_combined['FECHA'].dt.day

columns_to_keep = ['año', 'mes', 'dia', 'HORA', 'NOM_ESTACION', 'O3']
df_combined = df_combined[columns_to_keep]

print("\nPrimeras filas de datos transformados:")
print(df_combined.head())

df_combined.dropna(subset=['O3'], inplace=True)

print("\nValores nulos después de eliminar filas con nulos en O3:")
print(df_combined.isnull().sum())

df_combined = df_combined.apply(lambda col: col.astype('category').cat.codes if col.dtypes == 'object' or col.dtypes.name == 'category' else col)

y = df_combined['O3']
X = df_combined.drop(['O3'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf_model = RandomForestClassifier()
lr_model = LogisticRegression(max_iter=1000)

rf_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predictions)
lr_accuracy = accuracy_score(y_test, lr_predictions)

print(f"\nPrecisión de RandomForest: {rf_accuracy * 100:.2f}%")
print(f"Precisión de LogisticRegression: {lr_accuracy * 100:.2f}%")

best_model = rf_model if rf_accuracy > lr_accuracy else lr_model
target_name = 'O3'
pickle_filename = f"{target_name}_model.pkl"

with open(pickle_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"\nModelo con mejor puntaje ({'RandomForest' if best_model == rf_model else 'Logistic Regression'}) guardado como '{pickle_filename}'.")


modelo con target CO

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

dfElche = pd.read_csv('../Procesamiento/Elche-limpio.csv', sep=';')
dfOrihuela = pd.read_csv('../Procesamiento/Orihuela-limpio.csv', sep=';')
dfTorrevieja = pd.read_csv('../Procesamiento/Torrevieja-limpio.csv', sep=';')

df_combined = pd.concat([dfElche, dfOrihuela, dfTorrevieja], ignore_index=True)

print(df_combined.tail(10))

print("\nValores nulos iniciales por columna:")
print(df_combined.isnull().sum())

if 'FECHA' in df_combined.columns:
    df_combined['FECHA'] = pd.to_datetime(df_combined['FECHA'], errors='coerce')

    df_combined['año'] = df_combined['FECHA'].dt.year
    df_combined['mes'] = df_combined['FECHA'].dt.month
    df_combined['dia'] = df_combined['FECHA'].dt.day

columns_to_keep = ['año', 'mes', 'dia', 'HORA', 'NOM_ESTACION', 'CO']
df_combined = df_combined[columns_to_keep]

print("\nPrimeras filas de datos transformados:")
print(df_combined.head())

df_combined.dropna(subset=['CO'], inplace=True)

print("\nValores nulos después de eliminar filas con nulos en CO:")
print(df_combined.isnull().sum())

df_combined = df_combined.apply(lambda col: col.astype('category').cat.codes if col.dtypes == 'object' or col.dtypes.name == 'category' else col)

y = df_combined['CO']
X = df_combined.drop(['CO'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf_model = RandomForestClassifier()
lr_model = LogisticRegression(max_iter=1000)

rf_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predictions)
lr_accuracy = accuracy_score(y_test, lr_predictions)

print(f"\nPrecisión de RandomForest: {rf_accuracy * 100:.2f}%")
print(f"Precisión de LogisticRegression: {lr_accuracy * 100:.2f}%")

best_model = rf_model if rf_accuracy > lr_accuracy else lr_model
target_name = 'CO'
pickle_filename = f"{target_name}_model.pkl"

with open(pickle_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"\nModelo con mejor puntaje ({'RandomForest' if best_model == rf_model else 'Logistic Regression'}) guardado como '{pickle_filename}'.")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

dfElche = pd.read_csv('../Procesamiento/Elche-limpio.csv', sep=';')
dfOrihuela = pd.read_csv('../Procesamiento/Orihuela-limpio.csv', sep=';')
dfTorrevieja = pd.read_csv('../Procesamiento/Torrevieja-limpio.csv', sep=';')

df_combined = pd.concat([dfElche, dfOrihuela, dfTorrevieja], ignore_index=True)

print(df_combined.tail(10))

print("\nValores nulos iniciales por columna:")
print(df_combined.isnull().sum())

if 'FECHA' in df_combined.columns:
    df_combined['FECHA'] = pd.to_datetime(df_combined['FECHA'], errors='coerce')

    df_combined['año'] = df_combined['FECHA'].dt.year
    df_combined['mes'] = df_combined['FECHA'].dt.month
    df_combined['dia'] = df_combined['FECHA'].dt.day

columns_to_keep = ['año', 'mes', 'dia', 'HORA', 'NOM_ESTACION', 'SO2']
df_combined = df_combined[columns_to_keep]

print("\nPrimeras filas de datos transformados:")
print(df_combined.head())

df_combined.dropna(subset=['SO2'], inplace=True)

print("\nValores nulos después de eliminar filas con nulos en SO2:")
print(df_combined.isnull().sum())

df_combined = df_combined.apply(lambda col: col.astype('category').cat.codes if col.dtypes == 'object' or col.dtypes.name == 'category' else col)

y = df_combined['SO2']
X = df_combined.drop(['SO2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf_model = RandomForestClassifier()
lr_model = LogisticRegression(max_iter=1000)

rf_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predictions)
lr_accuracy = accuracy_score(y_test, lr_predictions)

print(f"\nPrecisión de RandomForest: {rf_accuracy * 100:.2f}%")
print(f"Precisión de LogisticRegression: {lr_accuracy * 100:.2f}%")

best_model = rf_model if rf_accuracy > lr_accuracy else lr_model
target_name = 'SO2'
pickle_filename = f"{target_name}_model.pkl"

with open(pickle_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"\nModelo con mejor puntaje ({'RandomForest' if best_model == rf_model else 'Logistic Regression'}) guardado como '{pickle_filename}'.")
